# Bokeh Plot

In [7]:
import numpy as np

from bokeh.io import output_notebook
from bokeh.layouts import column, gridplot
from bokeh.plotting import figure, curdoc, show, output_file, save
from bokeh.models import ColumnDataSource, Slider, CustomJS, Label

from GreenFunctions_SC import GSC_matrix, FSC_matrix
from GreenFunctions_MTI import GMTI_normalBC, GMTI_modifiedBC, GMTI_analytical
from Perturbation_Theory import GMTI2_analytical, GMTI2_modifiedBC, FMTI2_analytical, FMTI2_modifiedBC


# coupling matrix
Gamma = np.matrix([[1,0],[0,1],[1,0],[0,1]])
# parameters
d = 100.; mu = 0.010; Delta = 0.005; omega = 0.0025; L = 0.005; z0 = 0.

# colors for plotting
colors = ["#0077FF", "#00CC00", "#FFD700", "#FF2200"]
# labels for curves
labels = [["$f_{\\uparrow +, \\uparrow +}$", "$f_{\\uparrow +, \downarrow +}$", "$f_{\\uparrow +, \\uparrow -}$", "$f_{\\uparrow +, \downarrow -}$"],
          ["$f_{\downarrow +, \\uparrow +}$", "$f_{\downarrow +, \downarrow +}$", "$f_{\downarrow +, \\uparrow -}$", "$f_{\downarrow +, \downarrow -}$"],
          ["$f_{\\uparrow -, \\uparrow +}$", "$f_{\\uparrow -, \downarrow +}$", "$f_{\\uparrow -, \\uparrow -}$", "$f_{\\uparrow -, \downarrow -}$"],
          ["$f_{\downarrow -, \\uparrow +}$", "$f_{\downarrow -, \downarrow +}$", "$f_{\downarrow -, \\uparrow -}$", "$f_{\downarrow -, \downarrow -}$"]]



# Enable Bokeh in Jupyter Notebook
output_notebook()

# save result as html
#output_file(filename="test.html", title="HTML interactive plot")


# real space lattice
x = np.linspace(0, d, 150)
# define Zeeman values
zeeman = np.arange(-0.010, 0.011, 0.001)

Flist = [ [] for _ in range(16) ]
# loop over Zeeman
for L in zeeman:
    
    # compute pairing on lattice
    FMTI2 = [FMTI2_analytical(d=d, z=z, Z=z, z0=z0, L=L, mu=mu, Delta=Delta, omega=omega, Gamma=Gamma) for z in x]

    # loop over components
    for irow in range(4):
        for icol in range(4):
            # add F2 to F_list
            Flist[4*irow + icol].append( np.array([abs(f2[irow][icol]) for f2 in FMTI2]) )

xlim = (-5, 105.); ylim = (0,0.5)
# create 4 figures
p1 = figure(width=400, height=300, x_range=xlim, y_range=ylim, x_axis_label="z=z'", y_axis_label="|f(z,z)|")
p2 = figure(width=400, height=300, x_range=(0, d), y_range=(0,0.5), x_axis_label="z=z'", y_axis_label="|f(z,z)|")
p3 = figure(width=400, height=300, x_range=(0, d), y_range=(0,0.5), x_axis_label="z=z'", y_axis_label="|f(z,z)|")
p4 = figure(width=400, height=300, x_range=(0, d), y_range=(0,0.5), x_axis_label="z=z'", y_axis_label="|f(z,z)|")

p = [p1,p2,p3,p4]

# set font size for writings
for pi in p:
    # axis labels
    pi.xaxis.axis_label_text_font_size = '14pt'
    pi.yaxis.axis_label_text_font_size = '14pt'
    # legend
    #pi.legend.label_text_font_size = '12pt'
    # ticks font size
    pi.xaxis.major_label_text_font_size = '11pt'
    pi.yaxis.major_label_text_font_size = '12pt'
   
source = ColumnDataSource(data={'x': x, 
                                    'y1': Flist[0][0], 'y2': Flist[1][0], 'y3': Flist[2][0], 'y4': Flist[3][0],
                                    'y5': Flist[4][0], 'y6': Flist[5][0], 'y7': Flist[6][0], 'y8': Flist[7][0],
                                    'y9': Flist[8][0], 'y10': Flist[9][0], 'y11': Flist[10][0], 'y12': Flist[11][0],
                                    'y13': Flist[12][0], 'y14': Flist[13][0], 'y15': Flist[14][0], 'y16': Flist[15][0]})

lw = 3
# plot first column
p1.line('x', 'y1', source=source, line_width=lw, color='blue', legend_label="cos(2πx)")
p1.line('x', 'y2', source=source, line_width=lw, color='green')
p1.line('x', 'y3', source=source, line_width=lw, color='gold')
p1.line('x', 'y4', source=source, line_width=lw, color='red')

# plot second column
p2.line('x', 'y5', source=source, line_width=lw, color='blue')
p2.line('x', 'y6', source=source, line_width=lw, color='green')
p2.line('x', 'y7', source=source, line_width=lw, color='gold')
p2.line('x', 'y8', source=source, line_width=lw, color='red')

# plot third column
p3.line('x', 'y9', source=source, line_width=lw, color='blue')
p3.line('x', 'y10', source=source, line_width=lw, color='green')
p3.line('x', 'y11', source=source, line_width=lw, color='gold')
p3.line('x', 'y12', source=source, line_width=lw, color='red')

# plot third column
p4.line('x', 'y13', source=source, line_width=lw, color='blue')
p4.line('x', 'y14', source=source, line_width=lw, color='green')
p4.line('x', 'y15', source=source, line_width=lw, color='gold')
p4.line('x', 'y16', source=source, line_width=lw, color='red')
    
# Create a Slider
slider = Slider(start=0, end=len(zeeman)-1, value=0, step=1, title="Zeeman")

# CustomJS callback for switching arrays
callback = CustomJS(args=dict(source=source, y_arrays=Flist, slider=slider), code="""
        var index = slider.value;
        var data = source.data;
        slider.title = "Scaled Value: " + (slider.value-10).toFixed(2) + " meV";        
        data['y1'] = y_arrays[0][index];
        data['y2'] = y_arrays[1][index];
        data['y3'] = y_arrays[2][index];
        data['y4'] = y_arrays[3][index];
        data['y5'] = y_arrays[4][index];
        data['y6'] = y_arrays[5][index];
        data['y7'] = y_arrays[6][index];
        data['y8'] = y_arrays[7][index];
        data['y9'] = y_arrays[8][index];
        data['y10'] = y_arrays[9][index];
        data['y11'] = y_arrays[10][index];
        data['y12'] = y_arrays[11][index];
        data['y13'] = y_arrays[12][index];
        data['y14'] = y_arrays[13][index];
        data['y15'] = y_arrays[14][index];
        data['y16'] = y_arrays[15][index];
        source.change.emit();
""")



# update values dinamically    
slider.js_on_change('value', callback)
# slider layout
curdoc().add_root(slider)

# Show the plot in Jupyter
#show(column(p, slider))

# set minimum spacing
margin = 25
p1.min_border = margin; p2.min_border = margin; p3.min_border = margin; p4.min_border = margin

show(column(slider, gridplot([[p1, p2], [p3, p4]])))

#save(column(slider, gridplot([[p1, p2], [p3, p4]])))

Loading BokehJS ...